In [1]:
! pip install -U spacy -q
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.1.123+-x86_64-with-glibc2.35
Python version   3.12.11                       
Pipelines        en_core_web_lg (3.8.0), en_core_web_sm (3.8.0)



In [2]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [7]:
import json
f = open('annotations (1).json')
TRAIN_DATA = json.load(f)

In [9]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 17/17 [00:00<00:00, 4404.15it/s]


In [11]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.45    0.00    0.00    0.00    0.00
 44     200        111.07   1669.27  100.00  100.00  100.00    1.00
 96     400          0.00      0.01  100.00  100.00  100.00    1.00
163     600          0.00      0.00  100.00  100.00  100.00    1.00
241     800          0.00      0.00  100.00  100.00  100.00    1.00
341    1000          0.00      0.00  100.00  100.00  100.00    1.00
441    1200          0.00      0.00  100.00  100.00  100.00    1.00
584    1400          0.00      0.00  100.00  100.00  100.00    1.00
784    1600          0.00      0.00 

In [13]:
nlp_ner = spacy.load("/content/model-best")

In [14]:
doc = nlp_ner('''Microsoft Corp. announced its Q1 2025 earnings on October 18, 2025, reporting total revenue
of $61.9 billion, a 12% increase from the previous year. The company’s Intelligent Cloud
segment contributed $26.7 billion, while Office 365 and LinkedIn together generated
$18.3 billion in revenue. Microsoft’s stock closed at $328.45 on the New York Stock Exchange
after the earnings announcement.

Apple Inc. revealed that its iPhone 16 sales reached 48 million units in the September 2025
quarter, boosting total revenue to $94.2 billion. The company posted a net income of
$24.6 billion, supported by strong demand in the Asia-Pacific region. Apple shares climbed
to $195.10 following the report.

Meanwhile, JPMorgan Chase reported Q3 2025 profits of $13.4 billion, driven by growth in
consumer banking and investment services. CEO Jamie Dimon noted that U.S. economic
uncertainty and Federal Reserve interest rate policies remain key factors to watch in the
upcoming December 2025 FOMC meeting. Analysts at Morgan Stanley forecast that banking
sector profits may grow 7% in 2026 if rates stabilize.''')


In [15]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter